In [5]:
'''

掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

'\n\n掛載Google硬碟\n安裝套件\n引用套件\nAPP應用準備\n消息準備\nhandler執行方法設計\n啟動應用\n\n'

In [6]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內
#colab 會把資料清空(google電腦),在開發階段,為了保存資料,我們會希望掛載google drive
#把資料都存在google drive>>>只有在開發階段中使用,未來存東西,不應該存在這裡

'''
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
'''

流程解析

1.引用套件
2.建置app主邏輯
*生成準備傳給用戶的消息(以後可以把消息存在資料庫,提取)
3.設置以後收到消息,要怎麼處理
4.運行app

'''


'\n\n流程解析\n\n1.引用套件\n2.建置app主邏輯\n*生成準備傳給用戶的消息(以後可以把消息存在資料庫,提取)\n3.設置以後收到消息,要怎麼處理\n4.運行app\n\n'

In [8]:
# 安裝套件
!pip install line-bot-sdk flask flask-ngrok #flask-ngrok用來跟外部溝通的套件(開發用http)

In [9]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [10]:
'''
建置主程序app

建置handler與 line_bot_api

準備好app >>> 準備handler(用來接受Line傳來的一切服務,順便做驗證,過濾信件)
>>> 準備line_bot_api 有些業務要跟Line溝通,都會透過line_bot_api , EX:回信、取個資
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件,裡面要帶入1.Channel_Access_token 2.Channel_Secret
line_bot_api = LineBotApi("NLuJuE98mu9aVvIIfWPrgfq9q5EoVZYP+yWK6W4O+UsYdS/GmMpUYFk7qEM2OrrDXikIV+ZRTA1V/BftphfFFqpzzA8m+ygvmWsZ7iCn0kxank+IemyqfWLa7kYok8WV5d+3cTUXJC24ndjiGG6nDgdB04t89/1O/w1cDnyilFU=") 
handler = WebhookHandler("b660a0127b1ab050edfe4f9d9c543b4c")

In [11]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發

當流量消息,丟給我們的ngrok.xxx/ (根目錄),就會觸發Line消息的處理流程
'''

# 啟動server對外接口，使Line能丟消息進來
#flask套件
@app.route("/", methods=['POST']) 
#這裡運作完後,要把line+/丟給line Developers>API>Webhook裡面 ,並且要加https(line要求),ngrok有提供https服務
def callback():
    # get X-Line-Signature header value
    #line的加密簽章,每一則消息都會有
    signature = request.headers['X-Line-Signature']

    # get request body as text
    #用戶傳來的內容
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log  #cloud shell有其他方式
    #存在GD上,一個ai-event.log檔案
    f = open("/content/drive/MyDrive/2021-AI/LineBot/1020event/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
      #把消息交給handler,他會看是什麼樣的Event,做不同的處理
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [12]:

'''
用戶關注 >>> FollowEvent
撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 回應用戶，歡迎用的文字消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

from linebot.models import(
    TextSendMessage, ImageSendMessage
)

# 告知handler，如果收到FollowEvent，則執行下面的方法
#line_bot_api套件
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    #用line_bot_api跟line溝通,取得個資 get_profile(event.source.user_id)
    #>>>>event是每次傳來的消息
    # source是來源
    # user_id 發消息的用戶id
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
     # 存在GD的user.txt
    with open("/content/drive/MyDrive/2021-AI/LineBot/1020event/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    
    # 回覆文字消息與圖片消息
    # 請line_bot_api回應消息給line,line轉發回去給用戶
    # line_bot_api.reply_message(reply_token)
    # 每一次用戶傳給line,line轉傳過來的時,附上臨時票券,我們必須拿這個票券,跟line回應
    # TextSendMessage("回應的文字")
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('安安，你的個資已被我紀錄了')]
    )
    #表示有用戶加入:
    #運作中出現:127.0.0.1 - - [20/Oct/2021 04:28:23] "POST / HTTP/1.1" 200 -
    #這個是用戶消息格式的print(body):{"destination":"U5307897c9c8ff435a32a1d50e95ee58e","events":[{"type":"follow","timestamp":1634704102232,"source":{"type":"user","userId":"Ud28913e45991f8c7531b94f85a9992a1"},"replyToken":"e5283a0049fd4b56b91c037b6a02d39e","mode":"active"}]}
    #displayname:用戶名,status:用戶狀態,picture:用戶照片
    #有存到ai-event.txt

In [13]:
# 運行主程序
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://eda1-35-229-174-244.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
{"destination":"U5307897c9c8ff435a32a1d50e95ee58e","events":[{"type":"follow","timestamp":1634704102232,"source":{"type":"user","userId":"Ud28913e45991f8c7531b94f85a9992a1"},"replyToken":"e5283a0049fd4b56b91c037b6a02d39e","mode":"active"}]}


127.0.0.1 - - [20/Oct/2021 04:28:23] "POST / HTTP/1.1" 200 -
